In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import gmaps
import os
from pprint import pprint
from scipy.stats import linregress
from config import api_key
from config import g_key

In [3]:
# Loan CSV file generated from WeatherPy Folder
weather_data_to_load = "../WeatherPy/output_data/cities.csv"
weather_data = pd.read_csv(weather_data_to_load)
weather_data.head(20)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,0,0,0,0,0,punta arenas,90,PH,1579714331,92,8.99,125.34,72.72,1.41
1,1,1,1,1,1,1,rikitea,0,PF,1579714332,80,-23.12,-134.97,78.06,12.53
2,2,2,2,2,2,2,atuona,33,PF,1579714332,75,-9.80,-139.03,80.82,13.38
3,3,3,3,3,3,3,skibbereen,75,IE,1579714108,87,51.55,-9.27,50.00,4.70
4,4,4,4,4,4,4,mwense,100,ZM,1579714332,98,-10.38,28.70,67.77,2.26
5,5,5,5,5,5,5,nikolskoye,75,RU,1579714333,61,59.70,30.79,32.00,15.66
6,6,6,6,6,6,6,avarua,100,CK,1579714333,88,-21.21,-159.78,80.60,11.41
7,7,7,7,7,7,7,jamestown,1,US,1579714333,62,42.10,-79.24,30.99,8.05
8,8,8,8,8,8,8,tuktoyaktuk,90,CA,1579714333,75,69.45,-133.04,-14.80,9.17
9,9,9,9,9,9,9,awbari,0,LY,1579714333,32,26.59,12.78,57.87,8.99


In [4]:
#cut unneeded columns
to_drop = ['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','Unnamed: 0.1.1.1','Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1']
weather_data.drop(to_drop, inplace=True, axis=1)
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,punta arenas,90,PH,1579714331,92,8.99,125.34,72.72,1.41
1,rikitea,0,PF,1579714332,80,-23.12,-134.97,78.06,12.53
2,atuona,33,PF,1579714332,75,-9.80,-139.03,80.82,13.38
3,skibbereen,75,IE,1579714108,87,51.55,-9.27,50.00,4.70
4,mwense,100,ZM,1579714332,98,-10.38,28.70,67.77,2.26
...,...,...,...,...,...,...,...,...,...
544,ambilobe,99,MG,1579715454,90,-13.20,49.05,78.62,9.78
545,tuatapere,54,NZ,1579715455,88,-46.13,167.68,57.00,11.99
546,itarema,93,BR,1579715317,59,-2.92,-39.92,88.02,10.60
547,nguru,0,NG,1579715455,16,12.88,10.45,74.28,12.37


In [6]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [8]:
# Define our map locations 
locations = weather_data[["Lat", "Lng"]].astype(float)

# Convert Humidity Rate to float and store
humidity_rate = weather_data["Humidity"].astype(float)

In [9]:

# Create a Humidity Heatmap layer
fig = gmaps.figure(map_type='HYBRID') 

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [11]:

# Narrow down Data Frame
Ideal_Temp = weather_data[(weather_data['Max Temp'] > 50) & (weather_data['Max Temp'] <= 62) ]

In [12]:

Ideal_Clouds = Ideal_Temp[(Ideal_Temp['Cloudiness'] > 80)]

In [13]:

Ideal_df = Ideal_Clouds[(Ideal_Clouds['Wind Speed'] < 15)]

In [14]:

Ideal_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
14,bluff,83,NZ,1579714334,88,-46.60,168.33,57.00,11.01
17,hobart,90,AU,1579714334,93,-42.88,147.33,57.20,5.82
37,fortuna,90,US,1579714014,66,40.60,-124.16,53.01,4.70
48,castro,100,BR,1579714338,97,-24.79,-50.01,59.77,7.09
57,ouro preto do oeste,100,BR,1579714339,96,-27.34,-51.61,57.92,7.14
65,kruisfontein,100,ZA,1579714415,92,-34.00,24.73,60.13,4.47
73,eureka,90,US,1579714567,66,40.80,-124.16,54.00,4.70
94,tiznit,82,MA,1579714569,72,29.58,-9.50,51.03,5.57
115,goroka,90,PG,1579714574,96,-6.08,145.39,58.01,3.71
134,puerto escondido,90,ES,1579714577,87,36.59,-6.23,51.80,8.05


In [15]:

# Add hotel name to data frame
hotels_df = Ideal_df.copy()
hotels_df["Hotel Name"] = ""
hotels_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
14,bluff,83,NZ,1579714334,88,-46.60,168.33,57.00,11.01,
17,hobart,90,AU,1579714334,93,-42.88,147.33,57.20,5.82,
37,fortuna,90,US,1579714014,66,40.60,-124.16,53.01,4.70,
48,castro,100,BR,1579714338,97,-24.79,-50.01,59.77,7.09,
57,ouro preto do oeste,100,BR,1579714339,96,-27.34,-51.61,57.92,7.14,
65,kruisfontein,100,ZA,1579714415,92,-34.00,24.73,60.13,4.47,
73,eureka,90,US,1579714567,66,40.80,-124.16,54.00,4.70,
94,tiznit,82,MA,1579714569,72,29.58,-9.50,51.03,5.57,
115,goroka,90,PG,1579714574,96,-6.08,145.39,58.01,3.71,
134,puerto escondido,90,ES,1579714577,87,36.59,-6.23,51.80,8.05,


In [21]:

# The following code was graciously given by Pablo

def get_hotel(lat,lng):
    # geocoordinates
    target_coordinates = f"{lat}, {lng}"
    target_radius = 5000
    target_type = "hotel"
    target_keyword = "hotel"
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "keyword": target_keyword,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    try:
        response = requests.get(base_url, params=params).json()
    except ValueError:
        response = ""
    return response

In [22]:
for city in hotels_df['City']:
    lat = hotels_df.loc[hotels_df['City'] == city]['Lat'].item()
    lng = hotels_df.loc[hotels_df['City'] == city]['Lng'].item()
    hotel_info = get_hotel(lat,lng)
    try:
        random_hotel = random.randint(0, len(hotel_info["results"])-1)
        hotel_name = hotel_info["results"][random_hotel]["name"]
    except ValueError:
            print (f"No hotel results for: {city}")
            hotel_name = "No Hotel Found"
    print(f"Finding hotels for: {city}, {hotel_name}!")
    hotels_df.loc[hotels_df['City'] == city, 'Hotel_Name']= hotel_name

/Users/philipharmon/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: `item` has been deprecated and will be removed in a future version
  
/Users/philipharmon/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: `item` has been deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


Finding hotels for: bluff, The Lighthouse Gallery!
Finding hotels for: hobart, Quest Waterfront!
Finding hotels for: fortuna, 6 Rivers Motel!
Finding hotels for: castro, Pousada Bela Vista!
Finding hotels for: ouro preto do oeste, Hotel Baiolim!
No hotel results for: kruisfontein
Finding hotels for: kruisfontein, No Hotel Found!
Finding hotels for: eureka, Red Lion Hotel Eureka!
No hotel results for: tiznit
Finding hotels for: tiznit, No Hotel Found!
Finding hotels for: goroka, EMMANUEL LODGE!
Finding hotels for: puerto escondido, Hotel Pinomar!
Finding hotels for: puerto ayora, Hotel Dunas Puerto!
Finding hotels for: palafrugell, Hotel BlauMar Llafranc!
Finding hotels for: burnie, Jones on Wilson!
Finding hotels for: apt, Longo Mai!
Finding hotels for: shingu, Gran Hotel!
Finding hotels for: warragul, Club Hotel!
Finding hotels for: tezu, Hotel Taboka!
Finding hotels for: linhai, Milan Fashion Hotel Linhai - Linhai!
Finding hotels for: ginir, Bale Ginir Tourist Hotel!
No hotel results

In [23]:

hotels_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel_Name
14,bluff,83,NZ,1579714334,88,-46.60,168.33,57.00,11.01,,The Lighthouse Gallery
17,hobart,90,AU,1579714334,93,-42.88,147.33,57.20,5.82,,Quest Waterfront
37,fortuna,90,US,1579714014,66,40.60,-124.16,53.01,4.70,,6 Rivers Motel
48,castro,100,BR,1579714338,97,-24.79,-50.01,59.77,7.09,,Pousada Bela Vista
57,ouro preto do oeste,100,BR,1579714339,96,-27.34,-51.61,57.92,7.14,,Hotel Baiolim
65,kruisfontein,100,ZA,1579714415,92,-34.00,24.73,60.13,4.47,,No Hotel Found
73,eureka,90,US,1579714567,66,40.80,-124.16,54.00,4.70,,Red Lion Hotel Eureka
94,tiznit,82,MA,1579714569,72,29.58,-9.50,51.03,5.57,,No Hotel Found
115,goroka,90,PG,1579714574,96,-6.08,145.39,58.01,3.71,,EMMANUEL LODGE
134,puerto escondido,90,ES,1579714577,87,36.59,-6.23,51.80,8.05,,Hotel Pinomar


In [24]:

hotels_df = hotels_df[hotels_df.Hotel_Name!= 'No Hotel Found']

In [25]:

hotels_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel_Name
14,bluff,83,NZ,1579714334,88,-46.60,168.33,57.00,11.01,,The Lighthouse Gallery
17,hobart,90,AU,1579714334,93,-42.88,147.33,57.20,5.82,,Quest Waterfront
37,fortuna,90,US,1579714014,66,40.60,-124.16,53.01,4.70,,6 Rivers Motel
48,castro,100,BR,1579714338,97,-24.79,-50.01,59.77,7.09,,Pousada Bela Vista
57,ouro preto do oeste,100,BR,1579714339,96,-27.34,-51.61,57.92,7.14,,Hotel Baiolim
73,eureka,90,US,1579714567,66,40.80,-124.16,54.00,4.70,,Red Lion Hotel Eureka
115,goroka,90,PG,1579714574,96,-6.08,145.39,58.01,3.71,,EMMANUEL LODGE
134,puerto escondido,90,ES,1579714577,87,36.59,-6.23,51.80,8.05,,Hotel Pinomar
155,puerto ayora,90,ES,1579714577,87,36.59,-6.23,51.80,8.05,,Hotel Dunas Puerto
226,palafrugell,90,ES,1579714888,93,41.92,3.16,57.00,11.41,,Hotel BlauMar Llafranc


In [26]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
humidity = hotels_df["Humidity"].astype(float)
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["Lat", "Lng"]].astype(float)

In [27]:

# Add marker layer ontop of heat map
fig = gmaps.figure(map_type='TERRAIN')
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
heatmap_layer = gmaps.heatmap_layer(locations,humidity) 
heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 20
fig.add_layer(heatmap_layer)
fig.add_layer(marker_layer)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))